
# Predict Raw Images with Pretrained Cloud Models

This file was created with ChatGPT.
This notebook loads a `.keras` model from the `best models/` folder and runs
predictions on all images found in `data/raw`.

**Usage:**

1. Put your test images into `data/raw/` (they can be in subfolders).
2. Make sure your desired model (e.g. `xception_head_best.keras`) is in `best models/`.
3. Set `MODEL_NAME` in the config cell below.
4. Run all cells.


In [ ]:

# =======================
# CONFIGURATION
# =======================

import os
import numpy as np
import cv2
from tensorflow import keras
from tensorflow.keras.applications.xception import preprocess_input

# Folder containing your saved models
MODEL_DIR = "best models"

# Folder where users drop their test images
RAW_DIR = os.path.join("data", "raw")

# Choose which model to use (file must exist in MODEL_DIR)
# Examples:
#   "xception_head_best.keras"
#   "xception_finetuned_top44.keras"
#   "xception_finetuned_top88.keras"
#   "xception_finetuned_top132.keras"
MODEL_NAME = "xception_head_best.keras"

# Class labels used during training
CLASS_NAMES = ["clear", "cloudy", "overcast"]

# Xception input size
IMG_SIZE = (299, 299)

print("MODEL_DIR:", MODEL_DIR)
print("RAW_DIR  :", RAW_DIR)
print("MODEL_NAME:", MODEL_NAME)


In [ ]:
# =======================
# HELPER FUNCTIONS
# =======================

def load_model(model_name: str):
    model_path = os.path.join(MODEL_DIR, model_name)
    if not os.path.isfile(model_path):
        raise FileNotFoundError(
            f"Model '{model_name}' not found in '{MODEL_DIR}'.\n"
            f"Expected path: {model_path}"
        )
    print(f"[INFO] Loading model: {model_path}")
    model = keras.models.load_model(model_path)
    print("[INFO] Model loaded.")
    return model


def iter_image_paths(root_dir: str):
    """Yield all image file paths under root_dir."""
    exts = (".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff")
    for dirpath, _, filenames in os.walk(root_dir):
        for fname in filenames:
            if fname.lower().endswith(exts):
                yield os.path.join(dirpath, fname)


def load_and_preprocess_image(path: str) -> np.ndarray:
    """Load image from disk and prepare it for Xception."""
    img = cv2.imread(path)
    if img is None:
        raise ValueError(f"Could not read image: {path}")

    # BGR -> RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Resize to model input size
    img = cv2.resize(img, IMG_SIZE, interpolation=cv2.INTER_AREA)

    img = img.astype("float32")
    img = preprocess_input(img)           # Xception preprocessing
    img = np.expand_dims(img, axis=0)     # Shape: (1, H, W, 3)
    return img


def predict_raw_folder(model):
    if not os.path.isdir(RAW_DIR):
        raise FileNotFoundError(
            f"RAW_DIR '{RAW_DIR}' does not exist. "
            "Create it and put some images inside."
        )

    image_paths = sorted(list(iter_image_paths(RAW_DIR)))
    if not image_paths:
        print(f"[WARN] No images found under '{RAW_DIR}'. Nothing to do.")
        return

    print(f"[INFO] Found {len(image_paths)} images in '{RAW_DIR}'.\n")
    print("=== Predictions ===")

    for path in image_paths:
        try:
            x = load_and_preprocess_image(path)
        except Exception as e:
            print(f"[ERROR] Skipping '{path}': {e}")
            continue

        probs = model.predict(x, verbose=0)[0]    # shape (num_classes,)
        pred_idx = int(np.argmax(probs))
        pred_label = CLASS_NAMES[pred_idx]
        confidence = float(probs[pred_idx])

        rel_path = os.path.relpath(path, RAW_DIR)
        print(f"{rel_path:40s} → {pred_label:9s} (p = {confidence:.3f})")


In [ ]:

# =======================
# RUN PREDICTIONS
# =======================

model = load_model(MODEL_NAME)
predict_raw_folder(model)
